## Практическое задание к уроку № 8 по теме "A/B-тестирование".

Взять датасет из google диска: https://drive.google.com/file/d/1MpWBFIbqu4mbiD0BBKYX6YhS-f4mN3Z_.  
Проверить гипотезу о том, в каком варианте теста (control/personalization) больше конверсия (converted) и значимо ли это отличие статистически.

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from statsmodels.stats import proportion

Импортируем датасет:

In [2]:
df = pd.read_csv('marketing_campaign.csv')
print(df.shape)
df.head()

(10037, 6)


,user_id,date_served,marketing_channel,variant,language_displayed,converted
0,a1000,1/1/18,House Ads,personalization,English,True
1,a1001,1/1/18,House Ads,personalization,English,True
2,a1002,1/1/18,House Ads,personalization,English,True
3,a1003,1/1/18,House Ads,personalization,English,True
4,a1004,1/1/18,House Ads,personalization,English,True


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10037 entries, 0 to 10036
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             10037 non-null  object
 1   date_served         10021 non-null  object
 2   marketing_channel   10022 non-null  object
 3   variant             10037 non-null  object
 4   language_displayed  10037 non-null  object
 5   converted           10022 non-null  object
dtypes: object(6)
memory usage: 470.6+ KB


Посмотрим количество пропусков:

In [4]:
df.isna().sum()

user_id                0
date_served           16
marketing_channel     15
variant                0
language_displayed     0
converted             15
dtype: int64

Заполним пропуски в нужном нам столбце *converted* модой:

In [5]:
df['converted'].fillna(df['converted'].value_counts().index[0], inplace=True)

Посмотрим соотношение значений в наших столбцах:

In [6]:
df['variant'].value_counts()

control            5091
personalization    4946
Name: variant, dtype: int64

*control* соответствует группе А, где мы не осуществляли воздействие,  
*personalization* соответствует группе Б, где воздействие осуществялось.

In [7]:
df['converted'].value_counts()

False    8961
True     1076
Name: converted, dtype: int64

Конвертируем строковые значения булевого признака в числовые:

In [8]:
df['converted'] = df['converted'].astype('int')

Посмотрим общее значение конверсии для обеих групп:

In [9]:
df['converted'].sum() / df.shape[0]

0.10720334761382884

Обозначим переменные для проведения статистического теста:

In [10]:
# Группа А
k1 = df[df['variant'] == 'control']['converted'].sum()
n1 = df[df['variant'] == 'control'].shape[0]

# Группа Б
k2 = df[df['variant'] == 'personalization']['converted'].sum()
n2 = df[df['variant'] == 'personalization'].shape[0]

k1, n1, k2, n2

(371, 5091, 705, 4946)

Конверсия группа А:

In [11]:
k1 / n1

0.07287369868395208

Конверсия группы Б:

In [12]:
k2 / n2

0.14253942579862516

Видим, что конверсия группы Б примерно в 2 раза превышает конверсию группы А.  
Посмотрим, является ли эта разница статистически значимой.  
Проведём z-тест и тест по критерию хи-квадрат:

In [13]:
z_score, z_pvalue = proportion.proportions_ztest(np.array([k1, k2]),
                                                 np.array([n1, n2]))
print(f'z_score = {z_score:.3f}, p-value = {z_pvalue:.3f}')

z_score = -11.279, p-value = 0.000


In [14]:
chisq, chi_pvalue, table = proportion.proportions_chisquare(np.array([k1, k2]),
                                                            np.array([n1, n2]))
print(f'chisq_score = {chisq:.3f}, p-value = {chi_pvalue:.3f}')

chisq_score = 127.213, p-value = 0.000


Оба теста показали, что нулевая гипотеза, которая говорит о том,  
что разница между получившимися конверсиями является случайной,  
с высокой долей вероятности является ложной.  
Таким образом считаем, что после применения персонализации конверсия  
изменилась значительно.

Попробуем также встроенный в библиотеку scipy двухвыборочный t-тест:

In [15]:
t_score, t_pvalue = stats.ttest_ind(df[df['variant'] == 'control']['converted'],
                                    df[df['variant'] == 'personalization']['converted'],
                                    equal_var=False)
print(f't_score = {t_score:.3f}, p-value = {t_pvalue:.3f}')

t_score = -11.303, p-value = 0.000


Получили аналогичный результат.